In [1]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [69]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import geopy.distance

import urllib.request
import json

In [3]:
db_name = "../database/e-commerce.db"

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    # YOUR CODE
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        return conn
    except Error as e:
        print(e)
        


conn = create_connection(db_name)
conn

2.6.0


In [4]:

    cur = conn.cursor()
#     the table wight_cm can become width_cm   --> order_delivered_carrier_date == order_delivered_customer_date
    cur.execute('''
                SELECT 
                order_item.item_id, 
                order_item.freight_value, 
                order_item.price, 
                weight_g,
                lenght_cm, 
                height_cm, wight_cm, 
                customer.zip_code, 
                seller.zip_code, 
                order_dataset.purchase_timestamp,
                order_dataset.order_estimated_delivery_date, 
                order_dataset.order_delivered_carrier_date
                FROM product
                LEFT JOIN order_item ON product.id = order_item.product_id
                LEFT JOIN order_dataset ON order_item.order_id = order_dataset.order_id
                LEFT JOIN customer ON customer.id = order_dataset.customer_id          
                LEFT JOIN seller ON order_item.seller_id = seller.id                
                ''')
    
    res_cur = cur.fetchall()
    

In [5]:
df_bdd = pd.DataFrame(res_cur, columns=["nb_product", "prix_de_livraison", "product_price", "weight", "lenght", "height", "width", "zip_code_customer", "zip_code_seller", "date_purchase", "date_delevery_estimate", "date_delivery"])
df_bdd

,nb_product,prix_de_livraison,product_price,weight,lenght,height,width,zip_code_customer,zip_code_seller,date_purchase,date_delevery_estimate,date_delivery
0,1,7.39,10.91,225.0,16.0,10.0,14.0,13848,3694,2018-04-24 16:16:53,2018-05-09 00:00:00,2018-04-27 16:42:17
1,1,17.99,248.00,1000.0,30.0,18.0,20.0,20551,2955,2018-01-31 18:55:36,2018-02-22 00:00:00,2018-02-16 13:27:08
2,1,7.82,79.80,154.0,18.0,9.0,15.0,5821,2016,2018-07-09 21:18:01,2018-07-18 00:00:00,2018-07-11 23:24:16
3,1,9.54,112.30,371.0,26.0,4.0,26.0,2018,13330,2018-08-03 08:55:50,2018-08-08 00:00:00,2018-08-06 15:14:32
4,1,8.29,37.90,625.0,20.0,17.0,13.0,5835,4461,2018-04-11 01:06:37,2018-04-26 00:00:00,2018-04-13 22:41:44
...,...,...,...,...,...,...,...,...,...,...,...,...
112645,2,16.11,29.90,700.0,31.0,13.0,20.0,38130,13720,2018-01-03 15:05:53,2018-02-02 00:00:00,2018-01-15 20:39:35
112646,1,23.28,29.90,700.0,31.0,13.0,20.0,46190,13720,2018-03-06 11:51:44,2018-04-05 00:00:00,2018-03-23 20:08:40
112647,2,23.28,29.90,700.0,31.0,13.0,20.0,46190,13720,2018-03-06 11:51:44,2018-04-05 00:00:00,2018-03-23 20:08:40
112648,3,23.28,29.90,700.0,31.0,13.0,20.0,46190,13720,2018-03-06 11:51:44,2018-04-05 00:00:00,2018-03-23 20:08:40


In [6]:
cur = conn.cursor()
#     the table wight_cm can become width_cm   --> order_delivered_carrier_date == order_delivered_customer_date
cur.execute('''
            SELECT 
            zip_code, latitude, longitude
            FROM geolocation
            ''')

res_cur = cur.fetchall()

df_geoloc = pd.DataFrame(res_cur, columns=["zip_code", "latitude", "longitude"])
df_geoloc


,zip_code,latitude,longitude
0,1037,-23.545621,-46.639292
1,1046,-23.546081,-46.644820
2,1046,-23.546129,-46.642951
3,1041,-23.544392,-46.639499
4,1035,-23.541578,-46.641607
...,...,...,...
1000158,99950,-28.068639,-52.010705
1000159,99900,-27.877125,-52.224882
1000160,99950,-28.071855,-52.014716
1000161,99980,-28.388932,-51.846871


In [7]:
df_bdd = df_bdd.drop_duplicates(['zip_code_customer']).merge(df_geoloc.drop_duplicates(['zip_code']), left_on='zip_code_customer', right_on='zip_code', how="left", suffixes = ("","_customer"))
df_bdd

,nb_product,prix_de_livraison,product_price,weight,lenght,height,width,zip_code_customer,zip_code_seller,date_purchase,date_delevery_estimate,date_delivery,zip_code,latitude,longitude
0,1,7.39,10.91,225.0,16.0,10.0,14.0,13848,3694,2018-04-24 16:16:53,2018-05-09 00:00:00,2018-04-27 16:42:17,13848.0,-22.331249,-46.922580
1,1,17.99,248.00,1000.0,30.0,18.0,20.0,20551,2955,2018-01-31 18:55:36,2018-02-22 00:00:00,2018-02-16 13:27:08,20551.0,-22.915371,-43.242996
2,1,7.82,79.80,154.0,18.0,9.0,15.0,5821,2016,2018-07-09 21:18:01,2018-07-18 00:00:00,2018-07-11 23:24:16,5821.0,-23.666423,-46.753278
3,1,9.54,112.30,371.0,26.0,4.0,26.0,2018,13330,2018-08-03 08:55:50,2018-08-08 00:00:00,2018-08-06 15:14:32,2018.0,-23.493511,-46.631067
4,1,8.29,37.90,625.0,20.0,17.0,13.0,5835,4461,2018-04-11 01:06:37,2018-04-26 00:00:00,2018-04-13 22:41:44,5835.0,-23.653137,-46.759223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14971,1,15.29,59.00,450.0,59.0,13.0,38.0,74786,7084,2018-04-27 13:33:36,2018-05-24 00:00:00,2018-05-08 22:52:07,74786.0,-16.637430,-49.174428
14972,1,10.96,30.97,443.0,21.0,13.0,17.0,5685,14090,2017-01-25 22:19:51,2017-02-20 00:00:00,2017-02-08 10:17:35,5685.0,-23.608422,-46.704976
14973,1,26.11,69.90,12300.0,40.0,40.0,40.0,36390,37564,2018-02-07 10:16:13,2018-03-06 00:00:00,2018-02-15 18:13:21,36390.0,-21.299180,-44.656737
14974,1,34.69,127.00,1400.0,27.0,7.0,27.0,65910,19470,2018-02-12 00:27:57,2018-03-15 00:00:00,2018-03-15 21:12:32,65910.0,-5.520220,-47.475720


In [8]:
df_bdd = df_bdd.drop_duplicates(['zip_code_seller']).merge(df_geoloc.drop_duplicates(['zip_code']), left_on='zip_code_seller', right_on='zip_code', how="left", suffixes = ("","_seller"))
df_bdd

,nb_product,prix_de_livraison,product_price,weight,lenght,height,width,zip_code_customer,zip_code_seller,date_purchase,date_delevery_estimate,date_delivery,zip_code,latitude,longitude,zip_code_seller,latitude_seller,longitude_seller
0,1,7.39,10.91,225.0,16.0,10.0,14.0,13848,3694,2018-04-24 16:16:53,2018-05-09 00:00:00,2018-04-27 16:42:17,13848.0,-22.331249,-46.922580,3694.0,-23.537922,-46.477696
1,1,17.99,248.00,1000.0,30.0,18.0,20.0,20551,2955,2018-01-31 18:55:36,2018-02-22 00:00:00,2018-02-16 13:27:08,20551.0,-22.915371,-43.242996,2955.0,-23.468441,-46.720688
2,1,7.82,79.80,154.0,18.0,9.0,15.0,5821,2016,2018-07-09 21:18:01,2018-07-18 00:00:00,2018-07-11 23:24:16,5821.0,-23.666423,-46.753278,2016.0,-23.500978,-46.627123
3,1,9.54,112.30,371.0,26.0,4.0,26.0,2018,13330,2018-08-03 08:55:50,2018-08-08 00:00:00,2018-08-06 15:14:32,2018.0,-23.493511,-46.631067,13330.0,-23.089035,-47.214419
4,1,8.29,37.90,625.0,20.0,17.0,13.0,5835,4461,2018-04-11 01:06:37,2018-04-26 00:00:00,2018-04-13 22:41:44,5835.0,-23.653137,-46.759223,4461.0,-23.692896,-46.680210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,1,26.03,169.99,3350.0,62.0,20.0,40.0,39113,8599,2018-01-26 14:36:46,2018-02-22 00:00:00,2018-02-27 19:16:48,39113.0,-17.694884,-43.333244,8599.0,-23.462597,-46.335317
1431,1,24.84,39.99,100.0,16.0,2.0,11.0,68926,9572,2017-02-28 22:12:55,2017-04-07 00:00:00,2017-03-16 13:42:06,68926.0,-0.053523,-51.160746,9572.0,-23.643648,-46.563861
1432,1,26.09,269.90,1133.0,30.0,18.0,23.0,58690,26125,2017-02-16 19:56:08,2017-03-29 00:00:00,None,58690.0,-7.379848,-36.942182,26125.0,-22.728758,-43.402930
1433,1,37.16,35.00,150.0,16.0,8.0,16.0,62748,5043,2018-07-15 10:46:14,2018-08-09 00:00:00,2018-07-27 20:43:29,62748.0,-4.467966,-38.901787,5043.0,-23.526016,-46.689754


In [9]:
df_bdd = df_bdd.rename({'latitude': 'latitude_customer', 'longitude': 'longitude_customer'}, axis=1)
df_bdd.drop(columns=['zip_code'])
df_bdd

,nb_product,prix_de_livraison,product_price,weight,lenght,height,width,zip_code_customer,zip_code_seller,date_purchase,date_delevery_estimate,date_delivery,zip_code,latitude_customer,longitude_customer,zip_code_seller,latitude_seller,longitude_seller
0,1,7.39,10.91,225.0,16.0,10.0,14.0,13848,3694,2018-04-24 16:16:53,2018-05-09 00:00:00,2018-04-27 16:42:17,13848.0,-22.331249,-46.922580,3694.0,-23.537922,-46.477696
1,1,17.99,248.00,1000.0,30.0,18.0,20.0,20551,2955,2018-01-31 18:55:36,2018-02-22 00:00:00,2018-02-16 13:27:08,20551.0,-22.915371,-43.242996,2955.0,-23.468441,-46.720688
2,1,7.82,79.80,154.0,18.0,9.0,15.0,5821,2016,2018-07-09 21:18:01,2018-07-18 00:00:00,2018-07-11 23:24:16,5821.0,-23.666423,-46.753278,2016.0,-23.500978,-46.627123
3,1,9.54,112.30,371.0,26.0,4.0,26.0,2018,13330,2018-08-03 08:55:50,2018-08-08 00:00:00,2018-08-06 15:14:32,2018.0,-23.493511,-46.631067,13330.0,-23.089035,-47.214419
4,1,8.29,37.90,625.0,20.0,17.0,13.0,5835,4461,2018-04-11 01:06:37,2018-04-26 00:00:00,2018-04-13 22:41:44,5835.0,-23.653137,-46.759223,4461.0,-23.692896,-46.680210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,1,26.03,169.99,3350.0,62.0,20.0,40.0,39113,8599,2018-01-26 14:36:46,2018-02-22 00:00:00,2018-02-27 19:16:48,39113.0,-17.694884,-43.333244,8599.0,-23.462597,-46.335317
1431,1,24.84,39.99,100.0,16.0,2.0,11.0,68926,9572,2017-02-28 22:12:55,2017-04-07 00:00:00,2017-03-16 13:42:06,68926.0,-0.053523,-51.160746,9572.0,-23.643648,-46.563861
1432,1,26.09,269.90,1133.0,30.0,18.0,23.0,58690,26125,2017-02-16 19:56:08,2017-03-29 00:00:00,None,58690.0,-7.379848,-36.942182,26125.0,-22.728758,-43.402930
1433,1,37.16,35.00,150.0,16.0,8.0,16.0,62748,5043,2018-07-15 10:46:14,2018-08-09 00:00:00,2018-07-27 20:43:29,62748.0,-4.467966,-38.901787,5043.0,-23.526016,-46.689754


## pour trouver la distance entre les geolocalisation nous tester plusieurs solutions.
### Solution 1:

In [10]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

lat1 = radians(-22.2569861454338)
lon1 = radians(-45.6968690671541)
lat2 = radians(-22.4733036420427)
lon2 = radians(-45.6139087768263)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
print("Should be:", 278.546, "km")

Result: 25.52941069384593
Should be: 278.546 km



### Solution 2:
### nous decidons de garder d'utiliser cette methode car plus simple et lisible.

In [11]:
coords_1 = (-22.2569861454338, -45.6968690671541)
coords_2 = (-22.4733036420427, -45.6139087768263)

print(geopy.distance.geodesic(coords_1, coords_2).km)

25.432299832611168


In [42]:
print(df_bdd[["latitude_customer","longitude_customer"]].to_string(index=False, header=None))

-22.331249 -46.922580
-22.915371 -43.242996
-23.666423 -46.753278
-23.493511 -46.631067
-23.653137 -46.759223
-20.415339 -42.891537
-23.460347 -46.540935
-23.699209 -46.396573
-12.784546 -38.390486
 -5.077309 -42.811761
-23.602163 -46.823716
-19.994603 -43.888394
-22.884746 -43.318324
 -7.977935 -34.918821
-23.517127 -46.190667
-23.494133 -46.970809
-22.930836 -45.455415
-23.656294 -46.554122
-21.146232 -43.762615
-23.462597 -46.335317
-23.323582 -46.571516
-27.615683 -48.587666
 -9.577174 -35.738418
-23.583598 -46.507148
-16.687527 -49.265462
-23.506891 -46.785866
-22.799804 -43.437372
 -7.982395 -34.919276
-23.548338 -46.841360
-18.167224 -42.766094
-26.471192 -48.998747
-21.530634 -42.192998
-19.939689 -43.975338
-22.842634 -43.074121
-23.520466 -46.705557
-22.598629 -44.035471
-15.840667 -48.051943
-23.557128 -46.604530
 -5.877772 -35.217160
-22.925733 -43.222347
 -6.101156 -38.204566
-23.246561 -45.897501
-23.598996 -46.683581
-17.736463 -49.804319
 -5.135516 -42.805422
-22.805776

In [102]:
distance = {"distance" : []
           }


for latitude_customer, longitude_customer, latitude_seller, longitude_seller in df_bdd[["latitude_customer","longitude_customer", "latitude_seller", "longitude_seller"]].to_numpy():
    coords_1 = (float(latitude_customer), float(longitude_customer))
    coords_2 = (float(latitude_seller), float(longitude_seller))
    try:
        res = geopy.distance.geodesic(coords_1, coords_2).km
        distance["distance"].append(res)
    except:
        distance["distance"].append(np.nan)
        print("error")
    
df_bdd["distance"] = distance["distance"]
df_bdd

# #     distance.append(geopy.distance.geodesic(coords_1, coords_2).km)
# for longitude, latitude in geoloc.items():
#     if longitude != 'nan' or latitude != 'nan':
#         print(longitude, latitude)




error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


,nb_product,prix_de_livraison,product_price,weight,lenght,height,width,zip_code_customer,zip_code_seller,date_purchase,date_delevery_estimate,date_delivery,zip_code,latitude_customer,longitude_customer,zip_code_seller,latitude_seller,longitude_seller,distance
0,1,7.39,10.91,225.0,16.0,10.0,14.0,13848,3694,2018-04-24 16:16:53,2018-05-09 00:00:00,2018-04-27 16:42:17,13848.0,-22.331249,-46.922580,3694.0,-23.537922,-46.477696,141.206907
1,1,17.99,248.00,1000.0,30.0,18.0,20.0,20551,2955,2018-01-31 18:55:36,2018-02-22 00:00:00,2018-02-16 13:27:08,20551.0,-22.915371,-43.242996,2955.0,-23.468441,-46.720688,361.255505
2,1,7.82,79.80,154.0,18.0,9.0,15.0,5821,2016,2018-07-09 21:18:01,2018-07-18 00:00:00,2018-07-11 23:24:16,5821.0,-23.666423,-46.753278,2016.0,-23.500978,-46.627123,22.395881
3,1,9.54,112.30,371.0,26.0,4.0,26.0,2018,13330,2018-08-03 08:55:50,2018-08-08 00:00:00,2018-08-06 15:14:32,2018.0,-23.493511,-46.631067,13330.0,-23.089035,-47.214419,74.618935
4,1,8.29,37.90,625.0,20.0,17.0,13.0,5835,4461,2018-04-11 01:06:37,2018-04-26 00:00:00,2018-04-13 22:41:44,5835.0,-23.653137,-46.759223,4461.0,-23.692896,-46.680210,9.184295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,1,26.03,169.99,3350.0,62.0,20.0,40.0,39113,8599,2018-01-26 14:36:46,2018-02-22 00:00:00,2018-02-27 19:16:48,39113.0,-17.694884,-43.333244,8599.0,-23.462597,-46.335317,711.055474
1431,1,24.84,39.99,100.0,16.0,2.0,11.0,68926,9572,2017-02-28 22:12:55,2017-04-07 00:00:00,2017-03-16 13:42:06,68926.0,-0.053523,-51.160746,9572.0,-23.643648,-46.563861,2656.777335
1432,1,26.09,269.90,1133.0,30.0,18.0,23.0,58690,26125,2017-02-16 19:56:08,2017-03-29 00:00:00,None,58690.0,-7.379848,-36.942182,26125.0,-22.728758,-43.402930,1834.043488
1433,1,37.16,35.00,150.0,16.0,8.0,16.0,62748,5043,2018-07-15 10:46:14,2018-08-09 00:00:00,2018-07-27 20:43:29,62748.0,-4.467966,-38.901787,5043.0,-23.526016,-46.689754,2268.677888


### on verifie que les valeurs inserer

In [106]:
print(geopy.distance.geodesic((-22.331249, -46.922580), (-23.537922, -46.477696)).km)
print(geopy.distance.geodesic((-22.915371, -43.242996), (-23.468441, -46.720688)).km)

141.20689555303554
361.2555347525922


In [ ]:

df_bdd.drop(columns=['zip_code'])